In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
from basenet.vgg16_bn import vgg16_bn, init_weights

class double_conv(nn.Module):
    def __init__(self, in_ch, mid_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch + mid_ch, mid_ch, kernel_size=1),
            nn.BatchNorm2d(mid_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)

class CRAFT(nn.Module):
    def __init__(self):
        super(CRAFT, self).__init__()
        self.basenet = vgg16_bn(pretrained=False, freeze=False)
        self.upconv1 = double_conv(1024, 512, 256)
        self.upconv2 = double_conv(512, 256, 128)
        self.upconv3 = double_conv(256, 128, 64)
        self.upconv4 = double_conv(128, 64, 32)

        self.conv_cls = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(32, 16, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, kernel_size=1), nn.ReLU(inplace=True),
            nn.Conv2d(16, 2, kernel_size=1)
        )

        init_weights(self.upconv1.modules())
        init_weights(self.upconv2.modules())
        init_weights(self.upconv3.modules())
        init_weights(self.upconv4.modules())
        init_weights(self.conv_cls.modules())

    def forward(self, x):
        sources = self.basenet(x)
        y = torch.cat([sources[0], sources[1]], dim=1)
        y = self.upconv1(y)
        y = F.interpolate(y, size=sources[2].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[2]], dim=1)
        y = self.upconv2(y)
        y = F.interpolate(y, size=sources[3].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[3]], dim=1)
        y = self.upconv3(y)
        y = F.interpolate(y, size=sources[4].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[4]], dim=1)
        feature = self.upconv4(y)
        y = self.conv_cls(feature)
        return y.permute(0, 2, 3, 1), feature

def load_weights(model, weight_path):
    state_dict = torch.load(weight_path, map_location='cpu')
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] if k.startswith("module.") else k
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)

if __name__ == '__main__':
    model = CRAFT()
    weight_path = "craft_mlt_25k.pth"
    load_weights(model, weight_path)
    model.cuda()
    model.eval()
    output, _ = model(torch.randn(1, 3, 768, 768).cuda())
    print(output.shape)



torch.Size([1, 384, 384, 2])


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import cv2
import numpy as np
import pytesseract
from basenet.vgg16_bn import vgg16_bn, init_weights
import imgproc
import craft_utils

class double_conv(nn.Module):
    def __init__(self, in_ch, mid_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch+mid_ch, mid_ch, kernel_size=1),
            nn.BatchNorm2d(mid_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

class CRAFT(nn.Module):
    def __init__(self):
        super(CRAFT, self).__init__()
        self.basenet = vgg16_bn(pretrained=False, freeze=False)
        self.upconv1 = double_conv(1024, 512, 256)
        self.upconv2 = double_conv(512, 256, 128)
        self.upconv3 = double_conv(256, 128, 64)
        self.upconv4 = double_conv(128, 64, 32)
        self.conv_cls = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(32, 16, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, kernel_size=1), nn.ReLU(inplace=True),
            nn.Conv2d(16, 2, kernel_size=1)
        )
        init_weights(self.upconv1.modules())
        init_weights(self.upconv2.modules())
        init_weights(self.upconv3.modules())
        init_weights(self.upconv4.modules())
        init_weights(self.conv_cls.modules())
    def forward(self, x):
        sources = self.basenet(x)
        y = torch.cat([sources[0], sources[1]], dim=1)
        y = self.upconv1(y)
        y = F.interpolate(y, size=sources[2].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[2]], dim=1)
        y = self.upconv2(y)
        y = F.interpolate(y, size=sources[3].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[3]], dim=1)
        y = self.upconv3(y)
        y = F.interpolate(y, size=sources[4].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[4]], dim=1)
        feature = self.upconv4(y)
        y = self.conv_cls(feature)
        return y.permute(0,2,3,1), feature

def load_weights(model, weight_path):
    state_dict = torch.load(weight_path, map_location='cpu')
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] if k.startswith("module.") else k
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)

if __name__=='__main__':
    model = CRAFT()
    load_weights(model, "craft_mlt_25k.pth")
    model.cuda()
    model.eval()
    image = cv2.imread("img_test_ocr.png")
    img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
    ratio_h = ratio_w = 1/target_ratio
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2,0,1).unsqueeze(0).cuda()
    with torch.no_grad():
        output, _ = model(x)
    score_text = output[0,:,:,0].cpu().data.numpy()
    score_link = output[0,:,:,1].cpu().data.numpy()
    boxes, polys = craft_utils.getDetBoxes(score_text, score_link, 0.7, 0.4, 0.4, False)
    boxes = craft_utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    img_out = image.copy()
    for box in boxes:
        box_np = np.array(box, dtype=np.int32).reshape((-1,1,2))
        cv2.polylines(img_out, [box_np], True, (0,255,0), 2)
        x_min = int(box_np[:,0,0].min())
        y_min = int(box_np[:,0,1].min())
        x_max = int(box_np[:,0,0].max())
        y_max = int(box_np[:,0,1].max())
        roi = image[y_min:y_max, x_min:x_max]
        txt = pytesseract.image_to_string(roi, config="--psm 6").strip()
        print(txt)


Built
for
all
delivery
fleets.
9:19
78%
Whether
fleet
of
small
you
manage
a
delivery
vehicles
large
of
network
0.2
or
a
aall
E
36th
drivers,
NavaFLEET
Solution
scales
our
to
needs.
your
Then
Park
Ave
Ss
fleet
in
real-time
Track
your
See
exact
locations
and
get
—)

NEXT
z
sg
z
minute-by-minute
updates
with

tu
Google
Maps.
=
z
s
5
2
Set
boundaries
and
get
alerts
w
notifi
for
Receive
cations
f=
S
2
3
‘SI
unauthorized
movements,
deviations,
and
route
more
Analyze
driver
behavior.
v
2
>
Monitor
speed,
braking,
and
idling
improve
safety
and
to
performance
3
minutes
Optimize
and
routes
Save
0.8
mi
8:52
PM
fuel.
Reduce
costs
with
smarter,
efficient
routes.
more
Drop-off
package
NavaFleet
leverages
Maps
for
Google
425
Charleston
Rd
precise
fleet
visibility
and
operational
efficiency.
user-friendly,
Its
scalable
interface
requires
minima

training.
Customize
and
alerts,
reports,
dashboards
suit
operations.
to
your
reducing
improving
safety,
and
costs,
enhancing
delivery
efficiency.


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import cv2
import numpy as np
import pytesseract
from basenet.vgg16_bn import vgg16_bn, init_weights
import imgproc
import craft_utils

import spacy
nlp = spacy.load("en_core_web_md")

class double_conv(nn.Module):
    def __init__(self, in_ch, mid_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch+mid_ch, mid_ch, kernel_size=1),
            nn.BatchNorm2d(mid_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

class CRAFT(nn.Module):
    def __init__(self):
        super(CRAFT, self).__init__()
        self.basenet = vgg16_bn(pretrained=False, freeze=False)
        self.upconv1 = double_conv(1024, 512, 256)
        self.upconv2 = double_conv(512, 256, 128)
        self.upconv3 = double_conv(256, 128, 64)
        self.upconv4 = double_conv(128, 64, 32)
        self.conv_cls = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(32, 16, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, kernel_size=1), nn.ReLU(inplace=True),
            nn.Conv2d(16, 2, kernel_size=1)
        )
        init_weights(self.upconv1.modules())
        init_weights(self.upconv2.modules())
        init_weights(self.upconv3.modules())
        init_weights(self.upconv4.modules())
        init_weights(self.conv_cls.modules())
    def forward(self, x):
        sources = self.basenet(x)
        y = torch.cat([sources[0], sources[1]], dim=1)
        y = self.upconv1(y)
        y = F.interpolate(y, size=sources[2].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[2]], dim=1)
        y = self.upconv2(y)
        y = F.interpolate(y, size=sources[3].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[3]], dim=1)
        y = self.upconv3(y)
        y = F.interpolate(y, size=sources[4].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[4]], dim=1)
        feature = self.upconv4(y)
        y = self.conv_cls(feature)
        return y.permute(0,2,3,1), feature

def load_weights(model, weight_path):
    state_dict = torch.load(weight_path, map_location='cpu')
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] if k.startswith("module.") else k
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)

if __name__=='__main__':
    model = CRAFT()
    load_weights(model, "craft_mlt_25k.pth")
    model.cuda()
    model.eval()
    image = cv2.imread("img_test_ocr.png")
    orig_image = image.copy()
    img_resized, target_ratio, _ = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
    ratio_h = ratio_w = 1/target_ratio
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2,0,1).unsqueeze(0).cuda()
    with torch.no_grad():
        output, _ = model(x)
    score_text = output[0,:,:,0].cpu().data.numpy()
    score_link = output[0,:,:,1].cpu().data.numpy()
    boxes, _ = craft_utils.getDetBoxes(score_text, score_link, 0.7, 0.4, 0.4, False)
    boxes = craft_utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    for box in boxes:
        box_np = np.array(box, dtype=np.int32).reshape((-1,1,2))
        cv2.polylines(orig_image, [box_np], True, (0,255,0), 2)
        x_min = int(box_np[:,0,0].min())
        y_min = int(box_np[:,0,1].min())
        x_max = int(box_np[:,0,0].max())
        y_max = int(box_np[:,0,1].max())
        roi = orig_image[y_min:y_max, x_min:x_max]
        data = pytesseract.image_to_data(roi, config="--psm 6", output_type=pytesseract.Output.DICT)
        n_words = len(data['text'])
        for i in range(n_words):
            if data['text'][i].strip() != "":
                word = data['text'][i].strip()
                w_x = int(data['left'][i]) + x_min
                w_y = int(data['top'][i]) + y_min
                w_w = int(data['width'][i])
                w_h = int(data['height'][i])
                cv2.rectangle(orig_image, (w_x, w_y), (w_x+w_w, w_y+w_h), (255,0,0), 1)
                print(word, ": ", (w_x, w_y, w_w, w_h))
    cv2.imshow("Result", orig_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


Built :  (348, 10, 52, 16)
Tor :  (407, 10, 32, 16)
delivery :  (480, 8, 91, 23)
fleets. :  (577, 9, 69, 17)
9:19 :  (30, 27, 23, 9)
78% :  (260, 28, 24, 9)
Whether :  (347, 53, 64, 12)
fleet! :  (571, 53, 32, 12)
lof :  (608, 53, 15, 12)
small :  (526, 54, 38, 11)
YOu :  (415, 56, 26, 12)
manage :  (448, 56, 60, 12)
id :  (513, 56, 7, 9)
delivery, :  (347, 72, 58, 16)
Vehicles :  (410, 72, 58, 12)
large :  (508, 72, 36, 16)
i :  (615, 72, 14, 12)
nerwork :  (550, 74, 59, 10)
0.2 :  (70, 74, 45, 23)
ai :  (474, 75, 14, 9)
S :  (495, 76, 5, 8)
fall :  (126, 82, 19, 15)
E :  (232, 83, 9, 13)
36th :  (246, 84, 36, 12)
drivers, :  (347, 88, 56, 17)
NavaFLeEeE :  (439, 92, 82, 12)
Tl :  (512, 90, 10, 15)
Solution :  (528, 92, 63, 12)
scales :  (598, 92, 43, 12)
oun :  (407, 95, 25, 9)
ite :  (646, 94, 14, 10)
Needs. :  (386, 111, 43, 12)
YOU :  (346, 114, 34, 13)
Then :  (31, 131, 34, 12)
Park :  (97, 131, 31, 12)
Ave :  (134, 131, 25, 12)
Ss :  (163, 131, 8, 12)
fleet :  (474, 150, 35, 12)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict
import cv2
import numpy as np
import pytesseract
from basenet.vgg16_bn import vgg16_bn, init_weights
import imgproc
import craft_utils

class double_conv(nn.Module):
    def __init__(self, in_ch, mid_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch+mid_ch, mid_ch, kernel_size=1),
            nn.BatchNorm2d(mid_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    def forward(self, x):
        return self.conv(x)

class CRAFT(nn.Module):
    def __init__(self):
        super(CRAFT, self).__init__()
        self.basenet = vgg16_bn(pretrained=False, freeze=False)
        self.upconv1 = double_conv(1024, 512, 256)
        self.upconv2 = double_conv(512, 256, 128)
        self.upconv3 = double_conv(256, 128, 64)
        self.upconv4 = double_conv(128, 64, 32)
        self.conv_cls = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(32, 16, kernel_size=3, padding=1), nn.ReLU(inplace=True),
            nn.Conv2d(16, 16, kernel_size=1), nn.ReLU(inplace=True),
            nn.Conv2d(16, 2, kernel_size=1)
        )
        init_weights(self.upconv1.modules())
        init_weights(self.upconv2.modules())
        init_weights(self.upconv3.modules())
        init_weights(self.upconv4.modules())
        init_weights(self.conv_cls.modules())
    def forward(self, x):
        sources = self.basenet(x)
        y = torch.cat([sources[0], sources[1]], dim=1)
        y = self.upconv1(y)
        y = F.interpolate(y, size=sources[2].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[2]], dim=1)
        y = self.upconv2(y)
        y = F.interpolate(y, size=sources[3].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[3]], dim=1)
        y = self.upconv3(y)
        y = F.interpolate(y, size=sources[4].size()[2:], mode='bilinear', align_corners=False)
        y = torch.cat([y, sources[4]], dim=1)
        feature = self.upconv4(y)
        y = self.conv_cls(feature)
        return y.permute(0,2,3,1), feature

def load_weights(model, weight_path):
    state_dict = torch.load(weight_path, map_location='cpu')
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:] if k.startswith("module.") else k
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)

def get_text_color(roi):
    roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(roi_gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    fg_color = np.mean(roi[binary == 0], axis=0)
    return fg_color

def merge_words_into_sentences(words, img_width, img_height):
    words = sorted(words, key=lambda w: (w["y"], w["x"]))
    sentences = []
    current_sentence = []
    threshold_x = 0.03 * img_width
    threshold_y = 0.03 * img_height
    for i in range(len(words)):
        if i == 0:
            current_sentence.append(words[i])
            continue
        prev_word = words[i - 1]
        current_word = words[i]
        x_gap = current_word["x"] - (prev_word["x"] + prev_word["w"])
        y_diff = abs(current_word["y"] - prev_word["y"])
        color_diff = np.linalg.norm(current_word["color"] - prev_word["color"])
        if x_gap < threshold_x and y_diff < threshold_y:
            current_sentence.append(current_word)
        else:
            sentences.append(current_sentence)
            for a in current_sentence:
                print(a['text'])
            current_sentence = [current_word]
    if current_sentence:
        sentences.append(current_sentence)
    return sentences


if __name__=='__main__':
    model = CRAFT()
    load_weights(model, "craft_mlt_25k.pth")
    model.cuda()
    model.eval()
    image = cv2.imread("img_test_ocr.png")
    img_resized, target_ratio, size_heatmap = imgproc.resize_aspect_ratio(image, 1280, interpolation=cv2.INTER_LINEAR, mag_ratio=1.5)
    ratio_h = ratio_w = 1/target_ratio
    x = imgproc.normalizeMeanVariance(img_resized)
    x = torch.from_numpy(x).permute(2,0,1).unsqueeze(0).cuda()
    with torch.no_grad():
        output, _ = model(x)
    score_text = output[0,:,:,0].cpu().data.numpy()
    score_link = output[0,:,:,1].cpu().data.numpy()
    boxes, polys = craft_utils.getDetBoxes(score_text, score_link, 0.7, 0.4, 0.4, False)
    boxes = craft_utils.adjustResultCoordinates(boxes, ratio_w, ratio_h)
    words = []
    for box in boxes:
        x_min, y_min = int(min(box, key=lambda p: p[0])[0]), int(min(box, key=lambda p: p[1])[1])
        x_max, y_max = int(max(box, key=lambda p: p[0])[0]), int(max(box, key=lambda p: p[1])[1])
        roi = image[y_min:y_max, x_min:x_max]
        text = pytesseract.image_to_string(roi, config="--psm 6").strip()
        if text:
            color = get_text_color(roi)
            words.append({"text": text, "x": x_min, "y": y_min, "w": x_max-x_min, "h": y_max-y_min, "color": color})
    sentences = merge_words_into_sentences(words, image.shape[1], image.shape[0])
    img_out = image.copy()
    for sentence in sentences:
        x_min = min(word["x"] for word in sentence)
        y_min = min(word["y"] for word in sentence)
        x_max = max(word["x"] + word["w"] for word in sentence)
        y_max = max(word["y"] + word["h"] for word in sentence)
        text = " ".join(word["text"] for word in sentence)
        cv2.rectangle(img_out, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    cv2.imwrite("result.jpg", img_out)


[{'text': 'delivery', 'x': 477, 'y': 5, 'w': 95, 'h': 28, 'color': array([112.72907154,  69.91171994,  25.60273973])}, {'text': 'fleets.', 'x': 574, 'y': 7, 'w': 74, 'h': 22, 'color': array([112.21560575,  69.93634497,  27.45995893])}, {'text': 'Built', 'x': 346, 'y': 8, 'w': 56, 'h': 20, 'color': array([114.4787472 ,  73.42058166,  32.87695749])}, {'text': 'for', 'x': 405, 'y': 8, 'w': 36, 'h': 20, 'color': array([112.13362069,  69.33189655,  25.5862069 ])}, {'text': 'all', 'x': 445, 'y': 8, 'w': 29, 'h': 20, 'color': array([123.04132231,  81.52066116,  37.99586777])}, {'text': '9:19', 'x': 28, 'y': 25, 'w': 26, 'h': 12, 'color': array([106.01834862,  64.05504587,  18.21100917])}]
[{'text': '78%', 'x': 258, 'y': 25, 'w': 27, 'h': 13, 'color': array([107.71428571,  66.2244898 ,  20.90204082])}]
[{'text': 'Whether', 'x': 345, 'y': 50, 'w': 67, 'h': 16, 'color': array([112.31274131, 109.27799228, 110.28957529])}]
[{'text': 'fleet', 'x': 568, 'y': 50, 'w': 36, 'h': 16, 'color': array([98.

In [1]:
import cv2
import pytesseract

# Đọc ảnh
image = cv2.imread("img_test_ocr.png")

# Chuyển ảnh thành văn bản
text = pytesseract.image_to_string(image, lang="eng")

print(text)

919 4

? 0.2 mi

Then 4 Park Ave S

3

3 minutes

0.8 mi- 8:52 PM

Drop-off package

Charleston Rd

| > WadisoMPAVERaNZRi
sony ved!

wed

Built for all delivery fleets. :

Whether you manage a small fleet of
delivery vehicles or a large network of
drivers, our NavaFLEET Solution scales to
your needs.

@ Track your fleet in real-time.
See exact locations and get
minute-by-minute updates with
Google Maps.

@ Set boundaries and get alerts.

Receive notifications for

unauthorized movements,

route deviations, and more.

9

Analyze driver behavior.
Monitor speed, braking, and
idling to improve safety and
performance.

id Optimize routes and save
fuel. Reduce costs with smarter,
more efficient routes.

NavaFleet leverages Google Maps for
precise fleet visibility and operational
efficiency. Its user-friendly, scalable
interface requires minimal training.
Customize alerts, reports, and
dashboards to suit your operations—
reducing costs, improving safety, and
enhancing delivery efficiency.

